### Write your own Needleman-Wunch alignment script
- The script should take as input a tab delineated scoring matrix file, and two fasta formatted sequence text files.
- The script should produce as output a text file containing an optimized fasta formatted alignment based on the algorithm and input scoring matrix. 

In [3]:
## Let's import os
import os
print(os.getcwd()) # this will get the current working directory where out input files are stored

/home/user


In [4]:
# Open the provided Fasta file ATU76697.fasta to check the format
Sequence = open("ATU76697.fasta", 'r')
print(Sequence.read())
Sequence.close()

>gi|1790923|gb|U76697.1|ATU76697 Arabidopsis thaliana transposon-like element Limpet1, genomic sequence
GGGAATGTTCCAATATAAGACACTTTAAACGTAAGTTTAGACAATATAGACACTTTCCAAGTTAGAGGCA
CTTTTCCTTCTTTTTGAAGGAAAACTTGACTTTTATACCTCTTAACTAAACAATCGAAAACAATAACTAA
ATATATATCTTAACCAAACAATTAAAAAAATAAAAGAATTTAGATACGTAGTTATTAATATAGACCATTA
GATTGAAAAATAAAAATTAAGATCTATGGCTGAGATTAAAGACAATAAATGGATTAATTTTTTGATGTTA
AAATCTGATTAGAAAAAGGTATTTCTCTTCGTCTCTAAAACTAAATCTCTCTCTCTAAAAAAACAATCGT
TTCTCCCTTTCTCCTTCCTGAAGATCGTTTTTTCATAAATCCATAGTAGTTTAAAAACGAAGCAGAGAGA
TGTTGAAAATCGTTTCTCATGAAATTAATCGATTATTCTCTGTGAAGTTCTTTAATCCACACAACTTTCC
TCATGAACATGATAATAGTAGTAAATGGAGGTTTTTCCTATGGTTACTCTAGACGAAGGAGGATCTCCTT
GTGTTGGACAGGTTTGTGATTTCTTTCCATGGATTAAAAAAATTTGATTGTTTGTTTATGATGAACGATT
CTTTGGCTACGGAAGAGTGTCATGGAGTTCTGGCGAATTCTTTGGCTATGTTTGGTGATTTCGTTTTTAA
TCAAGTTGGGAATCAATAGGAAACAACTAAGCATACAACATAGATTAGAAGAGATATCAAGATGGATCTA
ATTTAAGTAAGATTTGGCGACTAATTCTAGATGATTAGGGTTATTTGTGATTTATTACAAGGCATTTGTG
TTCTCATTGATTTGGCGAGTAATTCTGTATGACTAGGGTTATTT

In [5]:
# Read the sequence_1 from a FASTA file ATU76697.fasta
def extract__transposonlikeelementsequence_from_fasta(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        sequence = ''.join(line.strip() for line in lines if not line.startswith('>'))
    return sequence

# Lets get our Sequence_1
file_path = "ATU76697.fasta"
seq1 = extract__transposonlikeelementsequence_from_fasta(file_path)
print("ATU76697 Sequence is:", seq1)
len_seq1 = len(seq1)

print("The length of the ATU76697 Sequence is:", len_seq1)

ATU76697 Sequence is: GGGAATGTTCCAATATAAGACACTTTAAACGTAAGTTTAGACAATATAGACACTTTCCAAGTTAGAGGCACTTTTCCTTCTTTTTGAAGGAAAACTTGACTTTTATACCTCTTAACTAAACAATCGAAAACAATAACTAAATATATATCTTAACCAAACAATTAAAAAAATAAAAGAATTTAGATACGTAGTTATTAATATAGACCATTAGATTGAAAAATAAAAATTAAGATCTATGGCTGAGATTAAAGACAATAAATGGATTAATTTTTTGATGTTAAAATCTGATTAGAAAAAGGTATTTCTCTTCGTCTCTAAAACTAAATCTCTCTCTCTAAAAAAACAATCGTTTCTCCCTTTCTCCTTCCTGAAGATCGTTTTTTCATAAATCCATAGTAGTTTAAAAACGAAGCAGAGAGATGTTGAAAATCGTTTCTCATGAAATTAATCGATTATTCTCTGTGAAGTTCTTTAATCCACACAACTTTCCTCATGAACATGATAATAGTAGTAAATGGAGGTTTTTCCTATGGTTACTCTAGACGAAGGAGGATCTCCTTGTGTTGGACAGGTTTGTGATTTCTTTCCATGGATTAAAAAAATTTGATTGTTTGTTTATGATGAACGATTCTTTGGCTACGGAAGAGTGTCATGGAGTTCTGGCGAATTCTTTGGCTATGTTTGGTGATTTCGTTTTTAATCAAGTTGGGAATCAATAGGAAACAACTAAGCATACAACATAGATTAGAAGAGATATCAAGATGGATCTAATTTAAGTAAGATTTGGCGACTAATTCTAGATGATTAGGGTTATTTGTGATTTATTACAAGGCATTTGTGTTCTCATTGATTTGGCGAGTAATTCTGTATGACTAGGGTTATTTGTGTTTTCTTAAAAAGAATTTGTGTTCTTGTTGAAATCTTGTTCATTGGAATTATTTGTGTTTGGTAAATCTTCATTGGTGGCTAAGGATGTGT

In [6]:
# Read the sequence_2 from a FASTA file LR782544.fasta
def extract_Arabidopsisthalianagenomeassemblysequence_from_fasta(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        sequence = ''.join(line.strip() for line in lines if not line.startswith('>'))
    return sequence

# Lets get our Sequence_2
file_path = "LR782544.fasta"
seq2 = extract_Arabidopsisthalianagenomeassemblysequence_from_fasta(file_path)
print("LR782544 Sequence is:", seq2)
len_seq2 = len(seq2)

print("The length of the LR782544 Sequence is:", len_seq2)

LR782544 Sequence is: GAGAATATTACACTCTTAAACACTTTTTTAAAACTATTTCTGTGTTTAGGCACTTTCAAAAGATTAGGCACTTTTTTATGTTTTTTTACTAAAATACCCTCTTAACTAAAATATTATCTTAACCCACGTTTTCTCTCTTTCTCTCTCTCTCTTCATCTCTCTCTTTCTCTCTCTCTCTCTCTTCATCTCTCTCTCTCTCTTCACCTCTCTCTCTCTTCATCTTTGTAAAAAATGAGAAGAAACTTCTTCTTCTTCAACAATGGCGGTTTGATTTGATTTTTGCTCTCTCTCATGTGTCAAACTCGTTTCATCTACGTAGCATCGAGATTTCAGTTAATCGAATCTCCTGTTATCATGGACATGTTATATAAAATGGATATAGATCAGTAAAAAGGTTTCCGAATGGATATATTTAAATGGATTCGTTATTATGGACACCAAAAAATTGTTTGTTGATTTTTTTTTCAGTTTGCAGAGACCTCCACCTTCAAAACCTTATGGACATGGAACTTTGGTTTCTTTACAAAACACGGTTAATCGAATCTCCTATTATTATTGACATGGTATATAGAATGGATACGGATCAGTAGAAAGGTTTCTGAATGGATATATTTAAATGGATTTGTTATTATGGATACCAAAAAATTATTTGTAGCAAATCAACCGCATATAGCTTGTTTTTAGGATGTTAGACATGCTTCTCCAATTCTTCATCTTTTACTCTTAACATGTAAGACCAAAAGATTTACAAATCTCACCATATCAATCCATAATATACAATCCATGATGCGTCATCCATAATATGTCATCCATGATTCATCATCCATAATGTGCCATCCATAATTTGCCATCCATAATGGACATCACCAATAACAAACGCCGTAAGAGCTACAAACACATCCTTAGCCACCAATGAAGATTTACCAAACACAAATAATTCCAATGAACAAGATTTCAACAAGAACACAAATTCTTTTT

In [8]:
matrix_values = open("blosum_example.txt", 'r')
substitution_matrix = {}

# Split the values into lines
lines = matrix_values.read().strip().split('\n')

# Iterate over lines to build the dictionary so that we can get the value in key- value format
for i in range(1, len(lines)):
    row_values = lines[i].split()
    Nucleic_acid1 = row_values[0]
    for j in range(1, len(row_values)):
        Nucleic_acid2 = lines[j][0] 
        score = int(row_values[j])
        substitution_matrix[(Nucleic_acid1, Nucleic_acid2)] = score

matrix_values.close()

In [9]:
# Declare the parameters
len_seq1 = len(seq1)
len_seq2 = len(seq2)
gap_penalty = -2
# Initialize the score matrix
score_matrix = []
for i in range(len_seq1+1):
    row = []
    for j in range(len_seq2+1):
        row.append(0)
    score_matrix.append(row)

In [10]:
# Initialize the first row and column with gap penalties
for i in range(1, len_seq1 + 1):
    score_matrix[i][0] = i * gap_penalty
    for j in range(1, len_seq2 + 1):
        score_matrix[0][j] = j * gap_penalty

In [11]:
# Initialize a counter for printing the error message
error_count = 0
# Fill the matrix with scores based on match, mismatch, and gap penalties
for i in range(1, len_seq1 + 1):
    for j in range(1, len_seq2 + 1):
        Nucleic_acid1 = seq1[i - 1]
        Nucleic_acid2 = seq2[j - 1]
        if (Nucleic_acid1, Nucleic_acid2) in substitution_matrix:
            match = score_matrix[i-1][j-1] + substitution_matrix[(Nucleic_acid1, Nucleic_acid2)]
            delete = score_matrix[i-1][j] + gap_penalty
            insert = score_matrix[i][j-1] + gap_penalty
            score_matrix[i][j] = max(match, delete, insert)
        else:
            # Print the error message only up to 2 times
            if error_count < 2: 
                print("Pair of Nucleotide " + Nucleic_acid1 + Nucleic_acid2 + " doesn't have a corresponding entry in your substitution matrix.")
                error_count += 1

Pair of Nucleotide AG doesn't have a corresponding entry in your substitution matrix.
Pair of Nucleotide AG doesn't have a corresponding entry in your substitution matrix.


In [12]:
# Initialize variables for traceback
i, j = len(seq1), len(seq2)
aligned_seq1 = []
aligned_seq2 = []

while i > 0 or j > 0:
    try:
        if i > 0 and j > 0 and score_matrix[i][j] == score_matrix[i-1][j-1] + substitution_matrix[(seq1[i-1], seq2[j-1])]:
            aligned_seq1.insert(0, seq1[i-1])
            aligned_seq2.insert(0, seq2[j-1])
            i -= 1
            j -= 1
        elif i > 0 and score_matrix[i][j] == score_matrix[i-1][j] + gap_penalty:
            aligned_seq1.insert(0, seq1[i-1])
            aligned_seq2.insert(0, '-')
            i -= 1
        else:
            aligned_seq1.insert(0, '-')
            aligned_seq2.insert(0, seq2[j-1])
            j -= 1
    except KeyError:
        # Handle KeyError, for example, by skipping the current iteration
        i -= 1
        j -= 1

aligned_seq1 = ''.join(aligned_seq1)
aligned_seq2 = ''.join(aligned_seq2)

print("Aligned Sequence 1:", aligned_seq1)
print("Aligned Sequence 2:", aligned_seq2)

Aligned Sequence 1: GGGAATGTTCCAATATAAGACACTTTAAACGTAAGTTTAGACAATATAGACACTTTCCAAGTTAGAGGCACTTTTCCTTCTTTTTGAAGGAAAACTTGACTTTTATACCTCTTAACTAAACAATCGAAAACAATAAAATA-TATA-TCTTA-ACCAAACA--ATTAAAAAAATAAAAG--AATTTAG-ATACG-T-A---G-TTATTAATATAG--ACCA---TTA-GATTGAAAAATAAAAATTAAGATCTATGGCTGAGATTAAAGACAATA-AATGGATTAATTTTTTG----ATGTTAAAATCTGATTAGAAAAAGGTATTTCTCTTCGTCTCTAAAACTAAATCTCTCTCTCTAAAAAAACAATCGTTTCTCCCTTTCTCCTTCCTGAAGATCGT-TTTTTCATAAATCCATAGTA------G-------------T-TT-AAAAACGAAG-CAG---AGA---GA-TGTTGAAAATCGTTTCTCATG--AAATTAATCG-ATTATTCTCTGTGAAGTTCTTTAATCCACACAACTTTCCTCATG----AACATG--ATAATAGT-AGTAAATGGAGGTTTTTCCTATGGTTA-CTCTA-G-ACG----------AA-GG--A-GGATCTCCTTG-TG------TTGGACAG----G-----TTTG--TG-ATTTCTTTCCATG--------GATTA--AAAAAATTTGATTGTTTG-TTTATGATGAACGATTCTTTG----GCTACGGAAGAGTGTCATGGAGTTCTGGCG-AATTCTTTG--G-CTATG----TTTGGTGATTTCGTTTTTA----ATCAAG-TTGGGAATCAATAGGAAACAACTA-----A--G-CATACAACATAG----ATTAGAAGAG-ATATCAAGATGGATCTAATTTAAGTAAG-----ATTTGGCGACTAATTCTAG-ATG-----A-TTA---GG----GTTATTTGTG-AT

In [13]:
# Write the alignment result to a text file

nw_file_path = "Global_alignment_results.txt"
with open(nw_file_path, 'w') as nw_alignment:
    # This line of code concatenates the string "Sequence 1: " with the content of aligned_seq1 and then adds a newline character ("\n")
    nw_alignment.write("Sequence 1: " + aligned_seq1 + "\n")
    nw_alignment.write("Sequence 2: " + aligned_seq2 + "\n")

print("Global Alignment results written to", nw_file_path)

Global Alignment results written to Global_alignment_results.txt


In [14]:
# Specify the file path
file_path = "NW_score_matrix.txt"

# Save the matrix to a text file
with open(file_path, "w") as file:
    for row in score_matrix:
        file.write(" ".join(map(str, row)) + "\n")

print(f"Score matrix for Global alignment has been saved to {file_path}")

Score matrix for Global alignment has been saved to NW_score_matrix.txt
